In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("awsaf49/coco-2017-dataset")

print("Path to dataset files:", path)

In [ ]:
import cv2
import os

def cartoonize_image(image_path, output_path):
    img = cv2.imread(image_path)

    # Étape 1 : Réduction des couleurs
    num_down = 2  
    num_bilateral = 7  
    img_down = img.copy()
    for _ in range(num_down):
        img_down = cv2.pyrDown(img_down)
    for _ in range(num_bilateral):
        img_down = cv2.bilateralFilter(img_down, d=9, sigmaColor=75, sigmaSpace=75)
    img_up = img_down
    for _ in range(num_down):
        img_up = cv2.pyrUp(img_up)

    # Resize img_up to match the original image size
    img_up = cv2.resize(img_up, (img.shape[1], img.shape[0]))


    # Étape 2 : Détection des contours
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray, 7)
    edges = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 2)

    # Étape 3 : Fusionner
    cartoon = cv2.bitwise_and(img_up, img_up, mask=edges)

    # Sauvegarde du résultat
    cv2.imwrite(output_path, cartoon)

# Exemple d'utilisation
input_dir = "dataset/real_images/"
output_dir = "dataset/cartoonized/"
os.makedirs(output_dir, exist_ok=True)

for image_file in os.listdir(input_dir):
    input_path = os.path.join(input_dir, image_file)
    output_path = os.path.join(output_dir, image_file)
    cartoonize_image(input_path, output_path)

In [ ]:
from sklearn.model_selection import train_test_split
import shutil

# Dossiers
real_images = "dataset/real_images"
cartoon_images = "dataset/cartoonized/"

train_real = "dataset/train/real/"
train_cartoon = "dataset/train/cartoonized/"
test_real = "dataset/test/real/"
test_cartoon = "dataset/test/cartoonized/"

os.makedirs(train_real, exist_ok=True)
os.makedirs(train_cartoon, exist_ok=True)
os.makedirs(test_real, exist_ok=True)
os.makedirs(test_cartoon, exist_ok=True)

# Récupération des fichiers
real_files = os.listdir(real_images)
cartoon_files = os.listdir(cartoon_images)

# Division
train_files, test_files = train_test_split(real_files, test_size=0.2, random_state=42)

# Copie des fichiers
for file in train_files:
    shutil.copy(os.path.join(real_images, file), os.path.join(train_real, file))
    shutil.copy(os.path.join(cartoon_images, file), os.path.join(train_cartoon, file))

for file in test_files:
    shutil.copy(os.path.join(real_images, file), os.path.join(test_real, file))
    shutil.copy(os.path.join(cartoon_images, file), os.path.join(test_cartoon, file))
